In [1]:
import os

In [2]:
%pwd

'd:\\Courses\\Krish_Naik\\Chicken_Disease_Classification\\research'

In [3]:
# move one folder up
os.chdir("../")

In [4]:
%pwd

'd:\\Courses\\Krish_Naik\\Chicken_Disease_Classification'

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

### 2. Update Entity

- Create a data class

In [7]:
from dataclasses import dataclass
from pathlib import Path

In [9]:
@dataclass(frozen = True)
class EvaluationConfig:
    model_path: Path
    training_data_path: Path
    all_params: dict
    params_image_size: int
    params_batch_size: int


### 3. Update Configuration Manager 

- Create root directory
- Create a sub directory
- initialize the data class object

In [11]:
from chicken_disease_classification.constants import *
from chicken_disease_classification.utils.common import read_yaml, create_directories, save_json

In [16]:
class ConfigurationManager:
    def __init__(
        self, 
        config_file_path: Path = CONFIG_FILE_PATH,
        params_file_path: Path = PARAMS_FILE_PATH
    ):
        # read base_model key in yaml file 
        self.config_file_path = read_yaml(config_file_path)
        # read params.yaml file
        self.params_file_path = read_yaml(params_file_path)
        
        # create directory -> "artifacts"; artifacts_root is present in config.yaml
        create_directories([self.config_file_path.artifacts_root])
    
    
    def get_validation_config(self) -> EvaluationConfig:
        
        # create obj of EvaluationConfig class
        eval_config_obj = EvaluationConfig(
            model_path = Path("artifacts/training/model.h5"),
            training_data_path = Path("artifacts/data_ingestion/Chicken-fecal-images"),
            all_params = self.params_file_path,
            params_image_size = self.params_file_path.IMAGE_SIZE,
            params_batch_size = self.params_file_path.BATCH_SIZE
        )
        
        return eval_config_obj

### 4. Create Components

In [14]:
from urllib.parse import urlparse

In [19]:
class Evaluation:
    
    def __init__(self, config: EvaluationConfig) -> None:
        self.config = config
    
    
    
    def _valid_generator(self):
        
        data_generator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        )
        
        dataflow_kwargs = dict(
            target_size = (self.config.params_image_size[:-1]),
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )
        
        
        valid_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            **data_generator_kwargs
        )
        
        self.valid_generator = valid_data_generator.flow_from_directory(
            directory = self.config.training_data_path,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )
    
    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    
    def evaluation(self):
        self.model = self.load_model(self.config.model_path)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
    
    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(data = scores, path_to_json = Path("scores.json"))
            

### 5 Create Pipeline

In [20]:
try:
    config = ConfigurationManager()
    validation_config = config.get_validation_config()
    evaluation_obj = Evaluation(config = validation_config)
    evaluation_obj.evaluation()
    evaluation_obj.save_score()
    
except Exception as e:
    raise e

[2023-06-10 14:21:59,112: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-06-10 14:21:59,117: INFO: common: yaml file: params.yaml loaded successfully]
[2023-06-10 14:21:59,121: INFO: common: Created directory at: artifacts]
Found 116 images belonging to 2 classes.
8/8 [==============================] - 17s 2s/step - loss: 0.7105 - accuracy: 0.9138
[2023-06-10 14:22:16,973: INFO: common: Saved json file at: scores.json]
